In [6]:
from PIL import Image
from lang_sam import LangSAM
import os
import numpy as np

model = LangSAM()

final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded from C:\Users\lukasz\.cache\huggingface\hub\models--ShilongLiu--GroundingDINO\snapshots\a94c9b567a2a374598f05c584e96798a170c56fb\groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])


In [7]:
input_base_dir = 'data/raw/'
output_base_dir = 'data/segmented/'

text_prompt = "animal"

subdirs = [d for d in os.listdir(input_base_dir) if os.path.isdir(os.path.join(input_base_dir, d)) and d != 'SPLIT3']

for subdir in subdirs:
    input_dir = os.path.join(input_base_dir, subdir)
    output_dir = os.path.join(output_base_dir, subdir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".jpg"):
            output_image_path = os.path.join(output_dir, filename)

            # Check if the file already exists in the output directory
            if os.path.exists(output_image_path):
                # print(f"Skipping {filename}, already segmented")
                continue

            # Read the image
            image_path = os.path.join(input_dir, filename)
            img_pil = Image.open(image_path).convert("RGB")

            masks, boxes, phrases, logits = model.predict(img_pil, text_prompt)

            if masks.dim() > 1:
                masks = masks.squeeze()
                if masks.dim() < 3:  # if masks is 2D
                    masks = masks.unsqueeze(0)  # make it 3D

                mask_combined = np.logical_or.reduce(masks.numpy())
                mask_3channel = np.repeat(mask_combined[:, :, np.newaxis], 3, axis=2)

                # Get the segmented dog image by multiplying (elementwise) the original image with the mask
                segmented_dog = np.array(img_pil) * mask_3channel

                # Create a new dark image with the same size
                dark_img = np.zeros_like(img_pil)

                # Paste the segmented dog onto the dark image
                final_img = dark_img + segmented_dog

                # Convert back to PIL image
                final_img_pil = Image.fromarray(final_img.astype(np.uint8))

                # Save the final image
                final_img_pil.save(output_image_path)
                print(filename + " saved")
            else:
                print("No cat detected in " + filename)